# Statistical models in R
This notebook is covering:
1. ARMA-models.

In [1]:
import pandas as pd
import plotly.express as px

In [2]:
# rpy2 is a Python package that allows you to run R code from Python
%pip install rpy2

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Load the rpy2 extension to use R in Jupyter
%load_ext rpy2.ipython

The magic function `%%R` is used for running R code in Jupyter

In [4]:
%%R
# Install required packages
if (require("dplyr") == FALSE) {
  install.packages("dplyr")
  library(dplyr)
}
if (require("zoo") == FALSE) {
  install.packages("zoo")
  library(zoo)
}
if (require("psych") == FALSE) {
  install.packages("psych")
  library(psych)
}
if (require("TSA") == FALSE) {
  install.packages("TSA")
  library(TSA)
}
if (require("forecast") == FALSE) {
  install.packages("forecast")
  library(forecast)
}
if (require("Metrics") == FALSE) {
  install.packages("Metrics")
  library(Metrics)
}
if (require("ggplot2") == FALSE) {
  install.packages("ggplot2")
  library(ggplot2)
}


Loading required package: dplyr

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Loading required package: psych
Loading required package: TSA

Attaching package: ‘TSA’

The following objects are masked from ‘package:stats’:

    acf, arima

The following object is masked from ‘package:utils’:

    tar

Loading required package: forecast
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 
Registered S3 methods overwritten by 'forecast':
  method       from
  fitted.Arima TSA 
  plot.Arima   TSA 
Loading required package: Metrics

Attaching package: ‘Metrics’

The following object is masked from ‘package:forecast’:

    accuracy

Loading required package: 

In [5]:
%%R
# Load data
hub_prices <- list(
  nbp = read.csv("../data/nbp_close.csv"),
  peg = read.csv("../data/peg_close.csv"),
  the = read.csv("../data/the_close.csv"),
  ttf = read.csv("../data/ttf_close.csv"),
  ztp = read.csv("../data/ztp_close.csv")
)

In [6]:
%%R
# Create a date index with a 5 day week to align data
start_date <- as.Date("2018-10-01")
end_date <- as.Date("2024-08-30")
date_seq <- seq.Date(start_date, end_date, by = "day")
date_index <- date_seq[!weekdays(date_seq) %in% c("Saturday", "Sunday")]

In [7]:
%%R
# Merge the hub data with the created date index so that all data is aligned and fill in missing values
hub_prices <- lapply(hub_prices, function(df) {
  df$Date <- as.Date(df$Date)
  df <- merge(data.frame(Date = date_index), df, by = "Date", all.x = TRUE)
  df <- df %>%
    mutate(CLOSE = na.approx(CLOSE, rule = 2))
  return(df)
})

In [8]:
%%R
# Load the prices processed pricing data for each hub
nbp_price <- hub_prices$nbp
peg_price <- hub_prices$peg
the_price <- hub_prices$the
ttf_price <- hub_prices$ttf
ztp_price <- hub_prices$ztp

In [9]:
%%R
# Calculate log returns for each hub
hub_returns <- lapply(hub_prices, function(df) {
  df <- df %>%
    mutate(Return = log(CLOSE) - lag(log(CLOSE))) %>%  # Calculate log returns
    slice(-1)  # Drop the first row
  return(df)
})

In [10]:
%%R
nbp_returns <- hub_returns$nbp
peg_returns <- hub_returns$peg
the_returns <- hub_returns$the
ttf_returns <- hub_returns$ttf
ztp_returns <- hub_returns$ztp

In [11]:
%%R
# Create a function to fit an ARMA with exogenous variables mode
armax_model <- function(hub1, hub2, ar_order, ma_order) {
  hub1 <- hub1 %>% slice(-1)
  hub2_lag1 <- lag(hub2$Return, 1) %>% slice(-1)
  armax_model <- arima(hub1$Return, order = c(ar_order, 0, ma_order), xreg = hub2_lag1$Return, optim.control = list(maxit = 1000))
return(armax_model)
}

In [45]:
%%R
head(ztp_returns)

        Date    CLOSE       Return
1 2018-10-02 27.23000  0.001837898
2 2018-10-03 27.10000 -0.004785579
3 2018-10-04 27.43000  0.012103577
4 2018-10-05 27.84000  0.014836531
5 2018-10-08 27.32333 -0.018732797
6 2018-10-09 26.80667 -0.019090425


In [12]:
%%R
expanding_window_armax_forecast <- function(hub1, hub2, ar_order, ma_order, window_size = 10) {
  
  n <- length(hub1$Return)
  
  start_points <- seq(n - 250, n - window_size, by = window_size)
  performance <- data.frame(Interval = numeric(), MAE = numeric(), RMSE = numeric())

  hub1 <- hub1 %>% slice(-1) # Drop the first row
  hub2_lag1 <- lag(hub2) %>% slice(-1) # Drop the first row

  all_predictions <- data.frame(
    Date = as.Date(character()),
    Interval = numeric(),
    Actual = numeric(),
    Predicted = numeric()
  )
  
  for (start in start_points) {
    
    hub1_train <- hub1[1:(start - 1), ]  # Use data up to the window for hub1
    hub2_train <- hub2[1:(start - 1), ]  # Use data up to the window for hub2
    hub2_lag1_train <- hub2_lag1[1:(start - 1), ]  # Use data up to the window for hub2
    
    hub1_actual <- hub1$Return[start:(start + window_size - 1)]

    armax_fit <- arima(hub1_train$Return, order = c(ar_order, 0, ma_order), xreg = hub2_lag1_train$Return, optim.control = list(maxit = 1000))
    # Fit the ARMA model on hub2
    hub2_arma <- arima(hub2_train$Return, order = c(3, 0, 3),  optim.control = list(maxit = 1000))  # Fit ARMA(3,3) on hub2

    last_value_hub2 <- tail(hub2_train$Return, 1)
    hub2_future_forecast <- c(last_value_hub2, predict(hub2_arma, n.ahead = window_size - 1)$pred) # Forecast the future values of hub2
 
    forecasted_values <- predict(armax_fit, newxreg = hub2_future_forecast, n.ahead = window_size)$pred
    
    mae_value <- mae(hub1_actual, forecasted_values)
    rmse_value <- rmse(hub1_actual, forecasted_values)
    performance <- rbind(performance, data.frame(Interval = start, MAE = mae_value, RMSE = rmse_value))

    prediction_data <- data.frame(
      Date = as.Date(hub1$Date[start:(start + window_size - 1)]),
      Interval = rep(start, window_size),
      Actual = hub1_actual,
      Predicted = as.numeric(forecasted_values)
    )
    all_predictions <- rbind(all_predictions, prediction_data)
  }
  
  return(list(performance = performance, predictions = all_predictions))
}

In [13]:
%%R
armax_performance_metrics <- expanding_window_armax_forecast(ztp_returns, ttf_returns, ar_order = 3, ma_order = 1, window_size = 5)
armax_predictions <- armax_performance_metrics$predictions
armax_actual_values <- armax_predictions$Actual
armax_forecasted_values <- armax_predictions$Predicted
mae_value <- mae(armax_actual_values, armax_forecasted_values)
rmse_value <- rmse(armax_actual_values, armax_forecasted_values)
print(paste0("Mean Absolute Error: ", mae_value))
print(paste0("Root Mean Squared Error: ", rmse_value))

[1] "Mean Absolute Error: 0.0304321917553044"
[1] "Root Mean Squared Error: 0.0392356475675944"


In [14]:
%R -o armax_predictions
armax_predictions = pd.DataFrame(armax_predictions)
armax_predictions['Date'] = pd.to_datetime(armax_predictions['Date'], origin='1970-01-01', unit='D')
fig = px.line(armax_predictions, x='Date', y=['Actual', 'Predicted'],
              labels={'value': 'Return', 'variable': 'Series'},
              title='Actual vs Predicted Returns')

fig.show()

In [65]:
%%R

N <- length(armax_actual_values)

ztp_close_prices <- tail(ztp_returns$CLOSE, N+1)

armax_actual_prices <- ztp_close_prices[1] * exp(cumsum(armax_actual_values))
armax_forecasted_prices <- ztp_close_prices[1] * exp(cumsum(armax_forecasted_values))


window_size <- 5

actual_prices <- c()
forecasted_prices <- c()


for (i in 1:(N - window_size)) {
  
  current_price <- ztp_close_prices[i]

  forecasted_price <- current_price * exp(sum(armax_forecasted_values[i:(i + window_size - 1)]))
  
  forecasted_prices <- c(forecasted_prices, forecasted_price)
  
  actual_price <- ztp_close_prices[i + window_size]

  actual_prices <- c(actual_prices, actual_price)
}

mae_price <- mae(actual_prices, forecasted_prices)
rmse_price <- rmse(actual_prices, forecasted_prices)

print(paste0("Mean Absolute Error for 5-Step Forecast: ", mae_price))
print(paste0("Root Mean Squared Error for 5-Step Forecast: ", rmse_price))


[1] "Mean Absolute Error for 5-Step Forecast: 2.23163366561969"
[1] "Root Mean Squared Error for 5-Step Forecast: 3.12349348258805"


In [67]:
%%R
forecast_comparison <- data.frame(
  Actual = actual_prices,
  Forecasted = forecasted_prices
)

In [68]:
# Import the %R magic and execute the R code
%R -o forecast_comparison

# Now price_comparison is available in Python
import plotly.graph_objs as go

# Create traces for actual and forecasted prices
actual_trace = go.Scatter(
    x=forecast_comparison.index,
    y=forecast_comparison['Actual'],
    mode='lines',
    name='Actual Prices'
)

forecasted_trace = go.Scatter(
    x=forecast_comparison.index,
    y=forecast_comparison['Forecasted'],
    mode='lines',
    name='Forecasted Prices'
)

# Create the plot
layout = go.Layout(
    title='Actual vs Forecasted Prices',
    xaxis=dict(title='Index'),
    yaxis=dict(title='Price'),
)

fig = go.Figure(data=[actual_trace, forecasted_trace], layout=layout)
fig.show()


In [15]:
%%R
expanding_window_arma_forecast <- function(hub1, ar_order, ma_order, window_size = 10) {
  
  n <- length(hub1$Return)
  
  start_points <- seq(n - 250, n - window_size, by = window_size)
  performance <- data.frame(Interval = numeric(), MAE = numeric(), RMSE = numeric())

  all_predictions <- data.frame(
    Date = as.Date(character()),
    Interval = numeric(),
    Actual = numeric(),
    Predicted = numeric()
  )
  
  for (start in start_points) {
    
    hub1_train <- hub1[1:(start - 1), ]  # Use data up to the window for hub1
    
    hub1_actual <- hub1$Return[start:(start + window_size - 1)]

    arma_fit <- arima(hub1_train$Return, order = c(ar_order, 0, ma_order), optim.control = list(maxit = 1000))

    forecasted_values <- predict(arma_fit, n.ahead = window_size)$pred
    
    mae_value <- mae(hub1_actual, forecasted_values)
    rmse_value <- rmse(hub1_actual, forecasted_values)
    performance <- rbind(performance, data.frame(Interval = start, MAE = mae_value, RMSE = rmse_value))
  

    prediction_data <- data.frame(
      Date = as.Date(hub1$Date[start:(start + window_size - 1)]),
      Interval = rep(start, window_size),
      Actual = hub1_actual,
      Predicted = as.numeric(forecasted_values)
    )

    all_predictions <- rbind(all_predictions, prediction_data)

  }
  
  return(list(performance = performance, predictions = all_predictions))

}

In [16]:
%%R
arma_performance_metrics <- expanding_window_arma_forecast(ztp_returns, ar_order = 3, ma_order = 3, window_size = 5)
arma_predictions <- arma_performance_metrics$predictions
arma_actual_values <- arma_predictions$Actual
arma_forecasted_values <- arma_predictions$Predicted
mae_value <- mae(arma_actual_values, arma_forecasted_values)
rmse_value <- rmse(arma_actual_values, arma_forecasted_values)
print(paste0("Mean Absolute Error: ", mae_value))
print(paste0("Root Mean Squared Error: ", rmse_value))

[1] "Mean Absolute Error: 0.0298737981139653"
[1] "Root Mean Squared Error: 0.0377958784099885"


In [17]:
%R -o arma_predictions
arma_predictions = pd.DataFrame(arma_predictions)
arma_predictions['Date'] = pd.to_datetime(arma_predictions['Date'], origin='1970-01-01', unit='D')
fig = px.line(arma_predictions, x='Date', y=['Actual', 'Predicted'],
              labels={'value': 'Return', 'variable': 'Series'},
              title='Actual vs Predicted Returns')

fig.show()

In [69]:
%%R

N <- length(arma_actual_values)

ztp_close_prices <- tail(ztp_returns$CLOSE, N+1)

armax_actual_prices <- ztp_close_prices[1] * exp(cumsum(arma_actual_values))
armax_forecasted_prices <- ztp_close_prices[1] * exp(cumsum(arma_forecasted_values))


window_size <- 5

actual_prices <- c()
forecasted_prices <- c()


for (i in 1:(N - window_size)) {
  
  current_price <- ztp_close_prices[i]

  forecasted_price <- current_price * exp(sum(arma_forecasted_values[i:(i + window_size - 1)]))
  
  forecasted_prices <- c(forecasted_prices, forecasted_price)
  
  actual_price <- ztp_close_prices[i + window_size]

  actual_prices <- c(actual_prices, actual_price)
}

mae_price <- mae(actual_prices, forecasted_prices)
rmse_price <- rmse(actual_prices, forecasted_prices)

print(paste0("Mean Absolute Error for 5-Step Forecast: ", mae_price))
print(paste0("Root Mean Squared Error for 5-Step Forecast: ", rmse_price))

[1] "Mean Absolute Error for 5-Step Forecast: 2.18332049690975"
[1] "Root Mean Squared Error for 5-Step Forecast: 3.01265673538633"


In [70]:
%%R
forecast_comparison <- data.frame(
  Actual = actual_prices,
  Forecasted = forecasted_prices
)

In [71]:
%R -o forecast_comparison

# Now price_comparison is available in Python
import plotly.graph_objs as go

# Create traces for actual and forecasted prices
actual_trace = go.Scatter(
    x=forecast_comparison.index,
    y=forecast_comparison['Actual'],
    mode='lines',
    name='Actual Prices'
)

forecasted_trace = go.Scatter(
    x=forecast_comparison.index,
    y=forecast_comparison['Forecasted'],
    mode='lines',
    name='Forecasted Prices'
)

# Create the plot
layout = go.Layout(
    title='Actual vs Forecasted Prices',
    xaxis=dict(title='Index'),
    yaxis=dict(title='Price'),
)

fig = go.Figure(data=[actual_trace, forecasted_trace], layout=layout)
fig.show()
